# Optimizing the parameters of each neuron of 2nn

In [14]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim

In [15]:
# Load Dataset (y = 3x + 2)
X = torch.tensor([[1.0], [2.0], [3.0], [4.0]])
y = torch.tensor([[5.0], [8.0], [11.0], [14.0]])

In [16]:
# One-layer neural network
class OneLayerNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(in_features=1, out_features=1)

    def forward(self, x):
        return self.linear(x)

model = OneLayerNN()

In [5]:
# Model with multiple layers
class MultiLayerNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 8)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(8, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.fc2(x)

model = MultiLayerNN()

Training setup

In [18]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(200):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

# Print parameters of each layer
print("Model parameters:\n")
for name, param in model.named_parameters():
    print(f"{name}:")
    print(param.data)
    print()

Model parameters:

linear.weight:
tensor([[3.1226]])

linear.bias:
tensor([1.6396])



## Access parameters of a specific layer only

In [6]:
# Option A: Direct access (most common)
print("fc1 weights:")
print(model.fc1.weight.data)

print("\nfc1 bias:")
print(model.fc1.bias.data)

fc1 weights:
tensor([[-0.1072],
        [-0.1632],
        [ 0.3657],
        [ 0.3105],
        [-0.0438],
        [-0.5763],
        [-0.5293],
        [-0.4856]])

fc1 bias:
tensor([-0.4895, -0.1649, -0.7619, -0.9934, -0.6422, -0.2573,  0.0135, -0.3514])


In [8]:
# Option B: Access via named_parameters()
# print("\nParameters belonging to fc2 only:\n")
for name, param in model.named_parameters():
    if name.startswith("fc2"):
        print(name)
        print(param.data)
        print()

fc2.weight
tensor([[-0.0207, -0.1601, -0.0975, -0.3399,  0.2201,  0.1136,  0.1165,  0.0932]])

fc2.bias
tensor([0.1404])



## Freeze or modify parameters of a layer

In [9]:
# Freeze parameters of fc1 layer
for param in model.fc1.parameters():
    param.requires_grad = False
# will not be updated during backprop
# fc2 will still train

In [10]:
# Modify parameters manually
#Example A: Manually set weights & bias (no gradients)
with torch.no_grad():
    model.fc2.weight.fill_(0.5)
    model.fc2.bias.fill_(0.0)

In [11]:
# Example B: Add noise to a layer’s weights
with torch.no_grad():
    model.fc1.weight += 0.01 * torch.randn_like(model.fc1.weight)

In [12]:
# Optimizer respects freezing
# When creating the optimizer after freezing, PyTorch automatically ignores frozen params:

optimizer = optim.SGD(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=0.01
)

In [13]:
# Verify which parameters are trainable
print("\nTrainable parameters:\n")
for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")


Trainable parameters:

fc1.weight: requires_grad=False
fc1.bias: requires_grad=False
fc2.weight: requires_grad=True
fc2.bias: requires_grad=True
